In [1]:
import numpy as np

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [2]:
class Module(object):
    """
    Basically, you can think of a module as of a something (black box)
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`:

        output = module.forward(input)

    The module should be able to perform a backward pass: to differentiate the `forward` function.
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule.

        gradInput = module.backward(input, gradOutput)
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True

    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.

        This includes
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput


    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.

        Make sure to both store the data in `output` field and return it.
        """

        # The easiest case:

        # self.output = input
        # return self.output

        pass

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input.
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.

        The shape of `gradInput` is always the same as the shape of `input`.

        Make sure to both store the gradients in `gradInput` field and return it.
        """

        # The easiest case:

        # self.gradInput = gradOutput
        # return self.gradInput

        pass

    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass

    def zeroGradParameters(self):
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass

    def getParameters(self):
        """
        Returns a list with its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def train(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True

    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [35]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially.

         `input` is processed by each module (layer) in self.modules consecutively.
         The resulting array is called `output`.
    """

    def __init__ (self):
        super(Sequential, self).__init__() 
        self.modules = []

    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)

    def updateGradInput(self, input, gradOutput):
        current_grad = gradOutput
    
        for i in reversed(range(len(self.modules))):
            prev_output = self.intermediate_outputs[i]
            current_grad = self.modules[i].updateGradInput(prev_output, current_grad)
            if hasattr(self.modules[i], "accGradParameters"):
                self.modules[i].accGradParameters(prev_output, current_grad)
        self.gradInput = current_grad
        return self.gradInput

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:

            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})


        Just write a little loop.
        """
        self.intermediate_outputs = [input]
        output = input
        for module in self.modules:
            output = module.updateOutput(output)
            self.intermediate_outputs.append(output)
        self.output = output
        return self.output
        
    

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:

            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)
            gradInput = module[0].backward(input, g_1)


        !!!

        To ech module you need to provide the input, module saw while forward pass,
        it is used while computing gradients.
        Make sure that the input for `i-th` layer the output of `module[i]` (just the same input as in forward pass)
        and NOT `input` to this Sequential module.

        !!!

        """
        return self.updateGradInput(input, gradOutput)

    def zeroGradParameters(self):
        for module in self.modules:
            module.zeroGradParameters()

    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]

    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]

    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string

    def __getitem__(self,x):
        return self.modules.__getitem__(x)

    def train(self):
        """
        Propagates training parameter through all modules
        """
        self.training = True
        for module in self.modules:
            module.train()

    def evaluate(self):
        """
        Propagates training parameter through all modules
        """
        self.training = False
        for module in self.modules:
            module.evaluate()

# Layers

## 1 (0.2). Linear transform layer
Also known as dense layer, fully-connected layer, FC-layer, InnerProductLayer (in caffe), affine transform
- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [4]:
class Linear(Module):
    """
    A module which applies a linear transformation
    A common name is fully-connected layer, InnerProductLayer in caffe.

    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()

        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)

        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = input @ self.W.T + self.b
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = gradOutput @ self.W
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradW = gradOutput.T @ input
        self.gradb = np.sum(gradOutput, axis=0)
        pass

    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

## 2. (0.2) SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

In [29]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()

    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        
        # softmax
        exp = np.exp(self.output)
        self.output = exp / np.sum(exp, axis=1, keepdims=True)
        
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Вычисление градиента softmax
        batch_size = input.shape[0]
        num_classes = input.shape[1]
        self.gradInput = np.zeros_like(input)
        
        for i in range(batch_size):
            s = self.output[i].reshape(-1, 1)  # -1 означает "автоматически определить размер" (C, 1)
            jacobian = np.diagflat(s) - np.dot(s, s.T)  # Матрица Якоби (C, C)
            self.gradInput[i] = np.dot(jacobian, gradOutput[i]) 
        
        return self.gradInput

    def __repr__(self):
        return "SoftMax"

## 3. (0.2) LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

In [6]:
class LogSoftMax(Module):
    def __init__(self):
         super(LogSoftMax, self).__init__()

    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))

        # Your code goes here. ################################################
        input = input - input.max(axis=1, keepdims=True)
        exp_sum = np.exp(input).sum(axis=1, keepdims=True)
        self.output = input - np.log(exp_sum)
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        exp_sum = np.exp(input).sum(axis=1, keepdims=True)
        self.gradInput = gradOutput - np.exp(input) * gradOutput.sum(axis=1, keepdims=True) / exp_sum
        return self.gradInput

    def __repr__(self):
        return "LogSoftMax"

## 4. (0.3) Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ - mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance:
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance.

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

In [7]:
class BatchNormalization(Module):
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = None
        self.moving_variance = None

    def updateOutput(self, input):
        # Your code goes here. ################################################
        # use self.EPS please
        if self.training:
            batch_mean = input.mean(axis=0)
            batch_var = input.var(axis=0)
            
            if self.moving_mean is None:
                self.moving_mean = batch_mean
                self.moving_variance = batch_var
            else:
                self.moving_mean = self.moving_mean * self.alpha + batch_mean * (1 - self.alpha)
                self.moving_variance = self.moving_variance * self.alpha + batch_var * (1 - self.alpha)
                
            self.output = (input - batch_mean) / np.sqrt(batch_var + self.EPS)
        else:
            self.output = (input - self.moving_mean) / np.sqrt(self.moving_variance + self.EPS)
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        batch_mean = input.mean(axis=0)
        batch_var = input.var(axis=0)
        
        n = input.shape[0]
        x_hat = (input - batch_mean) / np.sqrt(batch_var + self.EPS)
        
        dvar = np.sum(gradOutput * (input - batch_mean) * (-0.5) * (batch_var + self.EPS)**(-1.5), axis=0)
        dmean = np.sum(gradOutput * (-1) / np.sqrt(batch_var + self.EPS), axis=0) + dvar * np.mean(-2 * (input - batch_mean), axis=0)
        
        self.gradInput = gradOutput / np.sqrt(batch_var + self.EPS) + dvar * 2 * (input - batch_mean) / n + dmean / n
        return self.gradInput

    def __repr__(self):
        return "BatchNormalization"

In [8]:
class ChannelwiseScaling(Module):
    """
       Implements linear transform of input y = \gamma * x + \beta
       where \gamma, \beta - learnable vectors of length x.shape[-1]
    """
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()

        stdv = 1./np.sqrt(n_out)
        self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
        self.beta = np.random.uniform(-stdv, stdv, size=n_out)

        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def updateOutput(self, input):
        self.output = input * self.gamma + self.beta
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * self.gamma
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        self.gradBeta = np.sum(gradOutput, axis=0)
        self.gradGamma = np.sum(gradOutput*input, axis=0)

    def zeroGradParameters(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)

    def getParameters(self):
        return [self.gamma, self.beta]

    def getGradParameters(self):
        return [self.gradGamma, self.gradBeta]

    def __repr__(self):
        return "ChannelwiseScaling"

Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

## 5. (0.3) Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [9]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()

        self.p = p
        self.mask = None

    def updateOutput(self, input):
        # Your code goes here. ################################################
        if self.training:
            self.mask = (np.random.rand(*input.shape) > self.p) / (1 - self.p)
            self.output = input * self.mask
        else:
            self.output = input
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        if self.training:
            self.gradInput = gradOutput * self.mask
        else:
            self.gradInput = gradOutput
        return self.gradInput

    def __repr__(self):
        return "Dropout"

#6. (2.0) Conv2d
Implement [**Conv2d**](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html). Use only this list of parameters: (in_channels, out_channels, kernel_size, stride, padding, bias, padding_mode) and fix dilation=1 and groups=1.

In [38]:
class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=0, bias=True, padding_mode='zeros'):
        super(Conv2d, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride if isinstance(stride, tuple) else (stride, stride)
        self.padding = padding if isinstance(padding, tuple) else (padding, padding)

        self.bias_enabled = bias
        self.padding_mode = padding_mode

        kh, kw = self.kernel_size
        fan_in = in_channels * kh * kw
        stdv = 1. / np.sqrt(fan_in)

        self.weight = np.random.uniform(-stdv, stdv, (out_channels, in_channels, kh, kw)).astype(np.float32)
        self.gradWeight = np.zeros_like(self.weight)

        if bias:
            self.bias = np.random.uniform(-stdv, stdv, out_channels).astype(np.float32)
            self.gradBias = np.zeros_like(self.bias)
        else:
            self.bias = None
            self.gradBias = None

    def pad_input(self, input):
        pad_h, pad_w = self.padding
    
        pad_mode = self.padding_mode
        if pad_mode == 'zeros':
            # zeros → constant with constant_values=0
            return np.pad(
                input,
                ((0, 0), (0, 0), (pad_h, pad_h), (pad_w, pad_w)),
                mode='constant',
                constant_values=0
            )
        elif pad_mode == 'replicate':
            pad_mode = 'edge'
        elif pad_mode not in ['reflect', 'edge', 'wrap']:
            raise ValueError(f"Unsupported padding_mode: {self.padding_mode}")
    
        # для edge, reflect, wrap → pad без constant_values
        return np.pad(
            input,
            ((0, 0), (0, 0), (pad_h, pad_h), (pad_w, pad_w)),
            mode=pad_mode
        )

    def updateOutput(self, input):
        self.input = input
        N, C, H, W = input.shape
        OC, IC, KH, KW = self.weight.shape
        SH, SW = self.stride

        x_padded = self.pad_input(input)
        self.input_padded = x_padded

        H_out = (H + 2 * self.padding[0] - KH) // SH + 1
        W_out = (W + 2 * self.padding[1] - KW) // SW + 1

        self.output = np.zeros((N, OC, H_out, W_out), dtype=np.float32)

        for n in range(N):
            for oc in range(OC):
                for i in range(H_out):
                    for j in range(W_out):
                        h_start = i * SH
                        w_start = j * SW
                        region = x_padded[n, :, h_start:h_start+KH, w_start:w_start+KW]
                        self.output[n, oc, i, j] = np.sum(region * self.weight[oc])
                        if self.bias_enabled:
                            self.output[n, oc, i, j] += self.bias[oc]

        return self.output

    def updateGradInput(self, input, gradOutput):
        N, C, H, W = input.shape
        OC, IC, KH, KW = self.weight.shape
        SH, SW = self.stride
        H_out, W_out = gradOutput.shape[2:]

        grad_input_padded = np.zeros_like(self.input_padded, dtype=np.float32)

        for n in range(N):
            for oc in range(OC):
                for i in range(H_out):
                    for j in range(W_out):
                        h_start = i * SH
                        w_start = j * SW
                        grad = gradOutput[n, oc, i, j]
                        grad_input_padded[n, :, h_start:h_start+KH, w_start:w_start+KW] += grad * self.weight[oc]

        if self.padding != (0, 0):
            ph, pw = self.padding
            self.gradInput = grad_input_padded[:, :, ph:-ph or None, pw:-pw or None]
        else:
            self.gradInput = grad_input_padded

        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        N, C, H, W = input.shape
        OC, IC, KH, KW = self.weight.shape
        SH, SW = self.stride
        H_out, W_out = gradOutput.shape[2:]

        self.gradWeight.fill(0)
        if self.bias_enabled:
            self.gradBias.fill(0)

        for n in range(N):
            for oc in range(OC):
                for i in range(H_out):
                    for j in range(W_out):
                        h_start = i * SH
                        w_start = j * SW
                        region = self.input_padded[n, :, h_start:h_start+KH, w_start:w_start+KW]
                        grad = gradOutput[n, oc, i, j]
                        self.gradWeight[oc] += grad * region
                        if self.bias_enabled:
                            self.gradBias[oc] += grad

    def __repr__(self):
        return "Conv2d"

#7. (0.5) Implement [**MaxPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) and [**AvgPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html). Use only parameters like kernel_size, stride, padding (negative infinity for maxpool and zero for avgpool) and other parameters fixed as in framework.

In [37]:
class MaxPool2d(Module):
    def __init__(self, kernel_size, stride, padding):
        super(MaxPool2d, self).__init__()

        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride if isinstance(stride, tuple) else (stride, stride)
        self.padding = padding if isinstance(padding, tuple) else (padding, padding)

    def updateOutput(self, input):
        self.input = input
        N, C, H, W = input.shape
        kH, kW = self.kernel_size
        sH, sW = self.stride
        pH, pW = self.padding

        # padding with -inf
        padded = np.pad(input, ((0, 0), (0, 0), (pH, pH), (pW, pW)),
                        mode='constant', constant_values=-np.inf)

        out_H = (H + 2 * pH - kH) // sH + 1
        out_W = (W + 2 * pW - kW) // sW + 1
        self.output = np.zeros((N, C, out_H, out_W), dtype=np.float32)

        self.max_indices = {}
        for n in range(N):
            for c in range(C):
                for i in range(out_H):
                    for j in range(out_W):
                        h_start = i * sH
                        w_start = j * sW
                        window = padded[n, c, h_start:h_start + kH, w_start:w_start + kW]
                        max_val = np.max(window)
                        self.output[n, c, i, j] = max_val

                        # Save the position of max value for backward
                        max_pos = np.unravel_index(np.argmax(window), window.shape)
                        self.max_indices[(n, c, i, j)] = (h_start + max_pos[0], w_start + max_pos[1])

        return self.output

    def updateGradInput(self, input, gradOutput):
        N, C, H, W = input.shape
        grad_input = np.zeros_like(input, dtype=np.float32)

        for (n, c, i, j), (h, w) in self.max_indices.items():
            grad_input[n, c, h, w] += gradOutput[n, c, i, j]

        self.gradInput = grad_input
        return self.gradInput

    def __repr__(self):
        return "MaxPool2d"

class AvgPool2d(Module):
    def __init__(self, kernel_size, stride, padding):
        super(AvgPool2d, self).__init__()

        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride if isinstance(stride, tuple) else (stride, stride)
        self.padding = padding if isinstance(padding, tuple) else (padding, padding)

    def updateOutput(self, input):
        self.input = input
        N, C, H, W = input.shape
        kH, kW = self.kernel_size
        sH, sW = self.stride
        pH, pW = self.padding

        padded = np.pad(input, ((0, 0), (0, 0), (pH, pH), (pW, pW)),
                        mode='constant', constant_values=0)

        out_H = (H + 2 * pH - kH) // sH + 1
        out_W = (W + 2 * pW - kW) // sW + 1
        self.output = np.zeros((N, C, out_H, out_W), dtype=np.float32)

        for n in range(N):
            for c in range(C):
                for i in range(out_H):
                    for j in range(out_W):
                        h_start = i * sH
                        w_start = j * sW
                        window = padded[n, c, h_start:h_start + kH, w_start:w_start + kW]
                        self.output[n, c, i, j] = np.mean(window)

        return self.output
        

    def updateGradInput(self, input, gradOutput):
        N, C, H, W = input.shape
        kH, kW = self.kernel_size
        sH, sW = self.stride
        pH, pW = self.padding

        grad_input_padded = np.zeros((N, C, H + 2 * pH, W + 2 * pW), dtype=np.float32)
        out_H, out_W = gradOutput.shape[2:]

        for n in range(N):
            for c in range(C):
                for i in range(out_H):
                    for j in range(out_W):
                        h_start = i * sH
                        w_start = j * sW
                        grad = gradOutput[n, c, i, j] / (kH * kW)
                        grad_input_padded[n, c, h_start:h_start + kH, w_start:w_start + kW] += grad

        # Remove padding
        self.gradInput = grad_input_padded[:, :, pH:pH + H, pW:pW + W]
        return self.gradInput

    def __repr__(self):
        return "AvgPool2d"

#8. (0.3) Implement **GlobalMaxPool2d** and **GlobalAvgPool2d**. They do not have testing and parameters are up to you but they must aggregate information within channels. Write test functions for these layers on your own.

In [36]:
class GlobalMaxPool2d(Module):
    def __init__(self):
        super(GlobalMaxPool2d, self).__init__()

    def updateOutput(self, input):
        self.input = input
        N, C, H, W = input.shape
        self.output = np.max(input, axis=(2, 3))  # shape: (N, C)

        self.max_mask = (input == self.output[:, :, None, None])
        return self.output

    def updateGradInput(self, input, gradOutput):
        N, C, H, W = input.shape
        grad = np.zeros_like(input, dtype=np.float32)

        grad += self.max_mask.astype(np.float32) * gradOutput[:, :, None, None]
        self.gradInput = grad
        return self.gradInput

    def __repr__(self):
        return "GlobalMaxPool2d"


class GlobalAvgPool2d(Module):
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()

    def updateOutput(self, input):
        self.input = input
        N, C, H, W = input.shape
        self.output = np.mean(input, axis=(2, 3))  # shape: (N, C)
        return self.output

    def updateGradInput(self, input, gradOutput):
        N, C, H, W = input.shape
        grad = np.ones_like(input, dtype=np.float32) * (gradOutput[:, :, None, None] / (H * W))
        self.gradInput = grad
        return self.gradInput

    def __repr__(self):
        return "GlobalAvgPool2d"


#9. (0.2) Implement [**Flatten**](https://pytorch.org/docs/stable/generated/torch.flatten.html)

In [26]:
class Flatten(Module):
    def __init__(self, start_dim=0, end_dim=-1):
        super(Flatten, self).__init__()

        self.start_dim = start_dim
        self.end_dim = end_dim

    def updateOutput(self, input):
        self.input_shape = input.shape
        input = input.astype(np.float32)

        dims = len(input.shape)
        start = self.start_dim if self.start_dim >= 0 else dims + self.start_dim
        end = self.end_dim if self.end_dim >= 0 else dims + self.end_dim

        before = input.shape[:start]
        flattened = np.prod(input.shape[start:end + 1])
        after = input.shape[end + 1:]
        self.output = input.reshape(*before, flattened, *after)

        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput.reshape(self.input_shape)
        return self.gradInput

    def __repr__(self):
        return "Flatten"

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**):

In [14]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()

    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput

    def __repr__(self):
        return "ReLU"

## 10. (0.1) Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope.

In [15]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()

        self.slope = slope

    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.where(input > 0, input, input * self.slope)
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = np.where(input > 0, gradOutput, gradOutput * self.slope)
        return self.gradInput

    def __repr__(self):
        return "LeakyReLU"

## 11. (0.1) ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

In [16]:
class ELU(Module):
    def __init__(self, alpha = 1.0):
        super(ELU, self).__init__()

        self.alpha = alpha

    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.where(input > 0, input, self.alpha * (np.exp(input) - 1))
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = np.where(input > 0, gradOutput, gradOutput * self.alpha * np.exp(input))
        return self.gradInput

    def __repr__(self):
        return "ELU"

## 12. (0.1) SoftPlus
Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

In [17]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()

    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.log(1 + np.exp(input))
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = gradOutput * (1 - 1 / (1 + np.exp(input)))
        return self.gradInput

    def __repr__(self):
        return "SoftPlus"

#13. (0.2) Gelu
Implement [**Gelu**](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html) activations.

In [40]:
class Gelu(Module):
    def __init__(self):
        super(Gelu, self).__init__()

    def updateOutput(self, input):
        self.input = input.astype(np.float32)
        x = self.input
        self.output = 0.5 * x * (1.0 + np.erf(x / np.sqrt(2))).astype(np.float32)
        return self.output

    def updateGradInput(self, input, gradOutput):
        x = self.input
        erf_term = np.erf(x / np.sqrt(2))
        exp_term = np.exp(-0.5 * x**2)
        sqrt_2pi = np.sqrt(2 * np.pi)

        dgelu = 0.5 * (1.0 + erf_term) + (x * exp_term) / sqrt_2pi
        self.gradInput = (gradOutput * dgelu).astype(np.float32)
        return self.gradInput

    def __repr__(self):
        return "Gelu"

# Criterions

Criterions are used to score the models answers.

In [19]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None

    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)

    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

In [20]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()

    def updateOutput(self, input, target):
        self.output = np.sum(np.power(input - target,2)) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

## 14. (0.2) Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula,
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**



In [24]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15
    def __init__(self):
        a = super(ClassNLLCriterionUnstable, self)
        super(ClassNLLCriterionUnstable, self).__init__()

    def updateOutput(self, input, target):
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        self.input = input_clamp
        self.target = target

        losses = -np.sum(target * np.log(input_clamp), axis=1) 
        self.output = np.mean(losses)
        return self.output

    def updateGradInput(self, input, target):
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        batch_size = input.shape[0]

        self.gradInput = -target / (input_clamp * batch_size)
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterionUnstable"

## 15. (0.3) Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

In [23]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()

    def updateOutput(self, input, target):
        self.input = input
        self.target = target
        losses = -np.sum(target * input, axis=1)  # shape: [B]
        self.output = np.mean(losses)  # scalar
        return self.output

    def updateGradInput(self, input, target):
        batch_size = input.shape[0]
        self.gradInput = -target / batch_size  # shape: [B, C]
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterion"

1-я часть задания: реализация слоев, лосей и функций активации - 5 баллов. \\
2-я часть задания: реализация моделей на своих классах. Что должно быть:
  1. Выберите оптимизатор и реализуйте его, чтоб он работал с вами классами. - 1 балл.
  2. Модель для задачи мультирегрессии на выбраных вами данных. Использовать FCNN, dropout, batchnorm, MSE. Пробуйте различные фукнции активации. Для первой модели попробуйте большую, среднюю и маленькую модель. - 1 балл.
  3. Модель для задачи мультиклассификации на MNIST. Использовать свёртки, макспулы, флэттэны, софтмаксы - 1 балла.
  4. Автоэнкодер для выбранных вами данных. Должен быть на свёртках и полносвязных слоях, дропаутах, батчнормах и тд. - 2 балла. \\

Дополнительно в оценке каждой модели будет учитываться:
1. Наличие правильно выбранной метрики и лосс функции.
2. Отрисовка графиков лосей и метрик на трейне-валидации. Проверка качества модели на тесте.
3. Наличие шедулера для lr.
4. Наличие вормапа.
5. Наличие механизма ранней остановки и сохранение лучшей модели.
6. Свитч лося (метрики) и оптимайзера.